In [95]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import translators as ts
import time
import math
import re

Using state Silesia server backend.


In [171]:
class Match:
    def __init__(self, home_team=None, away_team=None, home_win=None,
                 draw=None, away_win=None, victory=None):
        self.home_team = home_team
        self.away_team = away_team
        self.home_win = home_win
        self.draw = draw
        self.away_win = away_win
        self.victory = victory
        self.match_page = None

    def get_upcoming_betting_odds(self, username=None, password=None):
        driver = webdriver.Chrome()

        driver.get("https://live.efortuna.pl/")

        driver.maximize_window()
        time.sleep(5)

        cookies_button = driver.find_element(By.ID, "cookie-consent-button-accept")
        cookies_button.click()

        login_button = driver.find_element(By.CSS_SELECTOR, "div.login_form__button")
        login_button.click()

        login_field = driver.find_element(By.NAME, "username")
        password_field = driver.find_element(By.NAME, "password")

        login_field.send_keys(username)
        password_field.send_keys(password)

        # final_login_button = driver.find_element(By.CSS_SELECTOR, "button.button.button-yellow.user-box-form-button")
        # final_login_button.click()

        expand_matches = driver.find_element(By.CLASS_NAME, "sport-header__button")
        expand_matches.click()

        actual_matches = driver.find_elements(By.CLASS_NAME, "live-match-info__team--1")
        for actual_match in actual_matches:
            if actual_match.get_attribute("innerHTML").__contains__(self.home_team):
                match_button = actual_match.find_element(By.XPATH, "./..")
                self.match_page = match_button.get_attribute("href")

        driver.close()
        self.bet_on_match()

    def bet_on_match(self):
        driver = webdriver.Chrome()

        driver.get(self.match_page)

        driver.maximize_window()
        time.sleep(5)

        cookies_button = driver.find_element(By.ID, "cookie-consent-button-accept")
        cookies_button.click()

        time.sleep(5)

In [172]:
match = Match(home_team="Chelsea")
# username = input("Username: ")
# password = input("Password: ")
# match.get_upcoming_betting_odds(username, password)
match.get_upcoming_betting_odds('username', 'password')

In [102]:
def get_past_results(matches_number=8, every=False):
    driver = webdriver.Chrome()

    driver.get("https://kickoff.ai/matches")

    driver.maximize_window()

    click_number = math.ceil(matches_number/8)
    for i in range(click_number):
        button = driver.find_element(By.CLASS_NAME, "more-matches-all")
        button.click()
        time.sleep(1)

    predictions = driver.find_elements(By.CLASS_NAME, "prediction-result")

    predictions = predictions[:matches_number]
    matches_arr = []
    for prediction in predictions:
        home_team = prediction.find_element(By.CSS_SELECTOR, '.team-home .team-name').get_attribute('innerHTML')
        home_team = ts.google(home_team, from_language='en', to_language='pl')

        away_team = prediction.find_element(By.CSS_SELECTOR, '.team-away .team-name').get_attribute('innerHTML')
        away_team = ts.google(away_team, from_language='en', to_language='pl')

        victory = "Draw"

        result = prediction.find_element(By.CLASS_NAME, "result").get_attribute("innerHTML")
        result_arr = re.findall(r'\d+', result)

        if int(result_arr[0]) > int(result_arr[1]):
            victory = home_team
        elif int(result_arr[0]) < int(result_arr[1]):
            victory = away_team

        bars = prediction.find_elements(By.CLASS_NAME, "progress-bar")
        temp_width_arr = []
        temp_sum = 0
        for bar in bars:
            temp_width_arr.append(bar.size['width'])
            temp_sum += bar.size['width']
        for i in range(len(temp_width_arr)):
            temp_width_arr[i] = round((temp_width_arr[i] / temp_sum) * 100)

        matches_arr.append(Match(home_team, away_team, temp_width_arr[0], temp_width_arr[1], temp_width_arr[2], victory))

    driver.close()
    return matches_arr

In [103]:
matches = get_past_results(2)

right_call = 0
wrong_call = 0

for match in matches:
    # print(f"In match: {i.home_team} vs {i.away_team}")
    # print(f"Home winning odds: {i.home_win}")
    # print(f"Draw odds: {i.draw}")
    # print(f"Away winning odds: {i.away_win}")
    # print(f"Team {i.victory} won!")

    print(f"W meczu {match.home_team} vs {match.away_team} / Zwycięzca: {match.victory}")
    odds = [match.home_win, match.draw, match.away_win]
    if max(odds) == match.home_win and max(odds) >= 50:
        if match.home_team != match.victory:
            print("-5 zl")
            wrong_call += 1
        else:
            print("Czysta 1 weszła")
            right_call += 1
    elif max(odds) == match.away_win and max(odds) >= 50:
        if match.away_team != match.victory:
            print("-5 zl")
            wrong_call += 1
        else:
            print("Czysta 2 weszła")
            right_call += 1
    elif max(odds) == match.home_win and max(odds) < 50:
        if (match.home_team != match.victory) and (match.victory != "Draw"):
            print("-5 zl")
            wrong_call += 1
        else:
            print("10 weszło")
            right_call += 1
    elif max(odds) == match.away_win and max(odds) < 50:
        if (match.away_team != match.victory) and (match.victory != "Draw"):
            print("-5 zl")
            wrong_call += 1
        else:
            print("02 weszło")
            right_call += 1

print(f"Dobrze: {right_call}, Źle: {wrong_call}")
print(f"Skuteczność: {right_call/(right_call + wrong_call)}")


W meczu Hiszpania vs Kostaryka / Zwycięzca: Hiszpania
Czysta 1 weszła
W meczu Niemcy vs Japonia / Zwycięzca: Japonia
-5 zl
Dobrze: 1, Źle: 1
Skuteczność: 0.5
